# 기본 구조 작성하기

### 아래의 그림 구조 만들어보기

1. 사용자 요청
2. 쿼리 3가지로 증강
3. gpt 단독 요청
4. 결과 relevant
5. rewrite query and 2번 반복
6. 종료

!["structure"](/home/ansgyqja/AI_application/images/relevant.png)

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agentic-RAG")

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('get_naver_news'), '..')))
from module.get_mcp_client import get_mcp_client
from module.get_news_origin import get_news_origin
from module.get_local_pdf import rag_pdf
from module.utils import convert_docs_str

In [ ]:
from typing import Annotated,Literal,List
from langchain_core.documents import Document
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import messages_to_history
from langchain_teddynote.evaluator import GroundednessChecker


In [ ]:
prompt = hub.pull("teddynote/rag-prompt-chat-history")

In [ ]:
# State 정의
class State(TypedDict):
    context: Annotated[str, 'Context']  # retrieve 검색 결과
    answer: Annotated[str, 'Answer'] # 사용자에게 제공하는 답변
    question: Annotated[str, "Question"] # 사용자 질의
    binary_score: Annotated[str, "Relevance"] # relevance 결과 yes or no
    chat_history:Annotated[list,add_messages]

def retrieve(state: State) -> State:
    # retrieve: 검색
    question = state['question']
    documents = rag_pdf().search_by_keyword(question)
    context = convert_docs_str(documents)
    return State({"context": context})
    
def gpt_generate(state: State) -> State:
    # gpt_request: 검색
    llm= ChatOpenAI(model='gpt-4.1-mini',temperature=0.5)
    chain = (
        {
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "chat_history": itemgetter("chat_history"),

        }
        | prompt
        | llm
        | StrOutputParser()
    )
    documents = chain.invoke(
        {
        "question": state['question'],
        "context": state['context'],
        "chat_history":messages_to_history(state['chat_history'])
        }
    )
    return {
            "answer": documents,
            "chat_history":[("user",state['question']),("assistant" , documents)]
            }

def claude_generate(state: State) -> State:
    # claude_request: 검색
    documents = "claude 답변"
    return {"answer": documents}

def relevance_check(state:State) -> State:
    question_retrieval_relevant = GroundednessChecker(
        llm=ChatOpenAI(model="gpt-4o-mini", temperature=0), target="question-retrieval"
    ).create()
    relevant = question_retrieval_relevant.invoke({"question":state["question"],"context":state["context"]})
    # gpt or claude response 검색
 
    return State({'binary_score':relevant.score})

def sum_answer(state:State)->State:
    # sum_answer 검색
    answer = "종합된 답변"
    return State(answer=answer)

def decision(state: State) -> Literal["rewrite_query",END]:
    # 의사결정
    # 로직을 추가할 수 가 있고요.
    
    if state["binary_score"] == "no" :
        return "rewrite_query"
    return END
    
def rewrite_query(state: State)->State:
    question = "새로운 질문"
    return State({'question':question})

In [ ]:


state_graph = StateGraph(State)

state_graph.add_node('retrieve',retrieve)
state_graph.add_node('gpt_generate',gpt_generate)
state_graph.add_node('gpt_relevance_check',relevance_check)
state_graph.add_node('rewrite_query',rewrite_query)



# state_graph.add_node('claude_generate',claude_generate)
# state_graph.add_node('claude_relevance_check',relevance_check)
# state_graph.add_node('sum_answer',sum_answer)


state_graph.add_edge(START,'retrieve')

state_graph.add_edge('retrieve','gpt_generate')
state_graph.add_edge('gpt_generate','gpt_relevance_check')
state_graph.add_conditional_edges(
    source='gpt_relevance_check',
    path=decision,
)
state_graph.add_edge('rewrite_query','retrieve')
state_graph.add_edge('gpt_generate',END)
# state_graph.add_edge('gpt_generate','gpt_relevance_check')
# state_graph.add_edge('gpt_relevance_check','sum_answer')

# state_graph.add_edge('retrieve','claude_generate')
# state_graph.add_edge('claude_generate','claude_relevance_check')
# state_graph.add_edge('claude_relevance_check','sum_answer')

# state_graph.add_conditional_edges(
#     source='sum_answer',
#     path=decision
# )
# state_graph.add_edge('rewrite_query','retrieve')


In [ ]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = state_graph.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=10,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "00"},  # 스레드 ID 설정
)

In [ ]:
question =  {"question":"오늘의 날씨"}
# for event in graph.stream(question,config=config,stream_mode="values"):
#     print("**"*30)
#     print(event)

In [ ]:
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

# invoke_graph(graph,question,config)

In [ ]:

stream_graph(graph,question,config)